In [13]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
import transformers
import torch
import torch.nn.functional as F
from torch import Tensor
from bert_score import score

In [24]:
!pip install transformers==4.42.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 48.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.18.1 requires transformers[sentencepiece]<4.40.0,>=4.26.0, but you have transformers 4.42.1 which is incompatible.


In [16]:
def parse_answers(answer):
    a_loc = answer.find('(A)')
    b_loc = answer.find('(B)')
    c_loc = answer.find('(C)')
    d_loc = answer.find('(D)')
    if a_loc == -1 and b_loc == -1 and c_loc == -1 and d_loc == -1:
        return 'Z'
    
    arr = [a_loc, b_loc, c_loc, d_loc]

    if 'the correct answer is (A)' in answer:
        return 'A'
    elif 'the correct answer is (B)' in answer:
        return 'B'
    elif 'the correct answer is (C)' in answer:
        return 'C'
    elif 'the correct answer is (D)' in answer:
        return 'D'
    
    if min(a_loc, b_loc, c_loc, d_loc) == a_loc:
        return 'A'
    elif min(a_loc, b_loc, c_loc, d_loc) ==  b_loc:
        return 'B'
    elif min(a_loc, b_loc, c_loc, d_loc) ==  c_loc:
        return 'C'
    elif min(a_loc, b_loc, c_loc, d_loc) ==  d_loc:
        return 'D'
    

In [17]:
df = pd.read_csv('../results/outputs/Llama3-Med42-70B_few-shot-cot_None_1.csv')
df.iloc[2]

Unnamed: 0                                                                        2
Source                                                           counselingexam.com
Exam Name                                                   NCMHCE Narrative Exam 1
Question #                                                                        3
Patient Demographic               Name: Alex Clinical Issues: Hopelessness/depre...
Mental Status Exam                Appearance: The client is dressed in a manner ...
Presenting Problem                First session You are a licensed mental health...
Question                          From a solution-focused therapy viewpoint, wha...
Choice A                          Ask the client to tell you about a time when h...
Choice B                          Ask the client to share what he has done in th...
Choice C                          Ask the client to rate his depression on a sca...
Choice D                                      Ask the client the "Miracle Qu

In [18]:
df['raw_pred'] = [x.replace("\\n", "") for x in df['raw_pred']]
df['Explanation for correct answer'].fillna("", inplace=True)
df['Explanation for correct answer'] = [x.replace("Explanation: ", "") for x in df['Explanation for correct answer']]
df['raw_pred'] = [x.replace("Explanation: ", "") for x in df['raw_pred']]
df['raw_pred'] = [x.strip() for x in df['raw_pred']]
df['raw_pred'] = [x[:x.find("\n")] for x in df['raw_pred']]

/tmp/ipykernel_1265492/188992109.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Explanation for correct answer'].fillna("", inplace=True)


In [19]:
# df.to_csv('../results/outputs/Meta-Llama-3-8B-Instruct_few-shot-cot_None_1.csv', index=False)

In [20]:
for idx, row in enumerate(df.iterrows()):
    print(idx)
    print(row[1]['Explanation for correct answer'])
    print()
    print(row[1]['raw_pred'])
    print()

0
When discussing the client's concerns and presentation during intake, it would be best to discuss suicidal ideation first. The client has been dealing with depression for a long time. Additionally, he expresses feelings of loneliness, disconnect from family members, difficulty in maintaining relationships with others, and a reliance on alcohol to manage his emotions. These factors can increase the risk of suicidal ideation or attempts, so it is important for the clinician to assess this issue with the client first.

Based on the given context and expert-level counseling knowledge, the primary focus for initial discussion should be (B) substance use history. The client specifically mentioned consuming drinks, particularly whiskey, both after work to calm down and in social settings. Considering his persistent depressive disorder, severe, and his expressed fatigue and withdrawal, it's crucial to explore whether his alcohol consumption may be a coping mechanism or if it exacerbates his 

In [21]:
def load_model(model_name):
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype = torch.float16, attn_implementation='flash_attention_2', device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

model, tokenizer = load_model('Salesforce/SFR-Embedding-Mistral')
x = tokenizer('Hello, my name is Johnny Smittens', max_length=4096, padding=True, truncation=True, return_tensors="pt")
x['input_ids'].shape[1]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of MistralForCausalLM were not initialized from the model checkpoint at Salesforce/SFR-Embedding-Mistral and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


11

In [22]:
def load_model(model_name):
    # 'Salesforce/SFR-Embedding-Mistral'
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")
    model = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="flash_attention_2", device_map="auto")
    return tokenizer, model

def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def generate_embedding(sentence, model, tokenizer):
    with torch.no_grad():
        batch_dict = tokenizer(sentence, max_length=4096, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**batch_dict)
        embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    return embeddings

def generate_embeddings_dataframe(df, tokenizer, model):
    embedding_dict = dict([])
    for idx, row in enumerate(df.iterrows()):
        embedding_pred = generate_embedding(row[1]['raw_pred'], model, tokenizer)
        embedding_true = generate_embedding(row[1]['Explanation for correct answer'], model, tokenizer)
        embedding_dict[idx] = (embedding_pred, embedding_true)
    return embedding_dict
        
def calculate_cosine_similarity(df, embedding_dict):
    similarity = []
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    for idx, row in enumerate(df.iterrows()):
        similarity.append(cos(embedding_dict[idx][0], embedding_dict[idx][1]).item())
    df['cosine_similarity'] = similarity
    return df

def calculate_bert_score(df):
    bert_score = []
    for idx, row in enumerate(df.iterrows()):
        P, R, F1 = score([row[1]['raw_pred']], [row[1]['Explanation for correct answer']], lang="en")
        bert_score.append(F1[0].item())
    df['bert_score'] = bert_score
    return df    

def calculate_length_difference(df, tokenizer):
    length_difference = []
    length_pred = []
    for idx, row in enumerate(df.iterrows()):
        e1 = tokenizer(row[1]['raw_pred'], max_length=4096, padding=True, truncation=True, return_tensors="pt")
        e2 = tokenizer(row[1]['Explanation for correct answer'], max_length=4096, padding=True, truncation=True, return_tensors="pt")
        length_difference.append(e1['input_ids'].shape[1] - e2['input_ids'].shape[1])
        length_pred.append(e1['input_ids'].shape[1])
    df['length_difference'] = length_difference
    df['length_prediction'] = length_pred
    return df

In [23]:
embeddings = generate_embeddings_dataframe(df, tokenizer, model)

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'last_hidden_state'

In [ ]:
print("Finished generating embeddings")
df = calculate_cosine_similarity(df, embeddings)
df = calculate_bert_score(df)
torch.cuda.empty_cache()

Finished generating embeddings


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [ ]:
df = calculate_length_difference(df, tokenizer)

In [ ]:
df

,Unnamed: 0,Source,Exam Name,Question #,Patient Demographic,Mental Status Exam,Presenting Problem,Question,Choice A,Choice B,...,Best answer,quote_rep,Answers,correct_answer_letter,raw_pred,parsed_pred,cosine_similarity,bert_score,length_difference,length_prediction
0,0,counselingexam.com,NCMHCE Narrative Exam 1,1,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What would you discuss first with the client b...,Method of dealing with conflict,Substance use history,...,False,False,(A): Method of dealing with conflict\n(B): Sub...,D,The client's primary concern is his depression...,B,0.863281,0.893880,-27,89
1,1,counselingexam.com,NCMHCE Narrative Exam 1,2,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What is irrelevant to consider when gathering ...,Is he averse to taking medication for depression?,How willing is he to change his profession?,...,False,False,(A): Is he averse to taking medication for dep...,B,The client's preference for whiskey is not rel...,Z,0.644531,0.853400,33,79
2,2,counselingexam.com,NCMHCE Narrative Exam 1,3,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,"From a solution-focused therapy viewpoint, wha...",Ask the client to tell you about a time when h...,Ask the client to share what he has done in th...,...,True,False,(A): Ask the client to tell you about a time w...,D,"The ""Miracle Question"" is a technique used in ...",D,0.981934,0.930018,12,78
3,3,counselingexam.com,NCMHCE Narrative Exam 1,4,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What is the best way to build rapport with the...,Assess the client's current level of support,Use humor to lighten the mood,...,True,False,(A): Assess the client's current level of supp...,C,This approach will help the client feel unders...,C,0.886719,0.875870,100,157
4,4,counselingexam.com,NCMHCE Narrative Exam 1,5,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,How would you approach the client's expressed ...,"""Do you think you won't have friends because d...","""Don't worry about your friends right now. Wha...",...,False,True,"(A): ""Do you think you won't have friends beca...",D,The best approach would be to empathize with t...,D,0.803711,0.880644,71,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,1616,tests.com,NCMHCE Practice Exam,562,51 Initial Intake: Age: 26 S...,":\nMolly was initially guarded, but pleasant, ...",Molly came into individual counseling due to i...,Risk factors for generalized anxiety disorder ...,Genetic loading,Respiratory disturbance,...,False,False,(A): Genetic loading\n(B): Respiratory disturb...,A,The risk factors for generalized anxiety disor...,A,0.852539,0.876777,-2,86
1617,1617,tests.com,NCMHCE Practice Exam,563,51 Initial Intake: Age: 26 S...,":\nMolly was initially guarded, but pleasant, ...",Molly came into individual counseling due to i...,The counselor code of ethics states that?,Bartering is not allowed.,Counselors should discontinue therapy for thos...,...,False,False,(A): Bartering is not allowed.\n(B): Counselor...,D,The counselor code of ethics states that barte...,A,0.911621,0.891085,-86,27
1618,1618,tests.com,NCMHCE Practice Exam,564,51 Initial Intake: Age: 26 S...,":\nMolly was initially guarded, but pleasant, ...",Molly came into individual counseling due to i...,Ineffective treatments of generalized anxiety ...,Benzodiazepine,Psychoeducation,...,False,False,(A): Benzodiazepine\n(B): Psychoeducation\n(C)...,D,The ineffective treatments for Generalized Anx...,D,0.9179

In [ ]:
import numpy as np
print(np.mean(df['cosine_similarity']))
print(np.mean(df['bert_score']))
print(np.mean(df['length_difference']))

0.8456371573295806
0.8749391129177194
-47.36520666255398
